<a href="https://colab.research.google.com/github/hidt4/python-compchem-book/blob/main/compchem_book_ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3章 Pythonで量子化学計算をはじめよう

### 環境構築

#### Google Colab上にPsi4をインストール
2023年8月時点では問題なく動作していましたが
今後のColabの開発動向次第では動作しなくなる可能性があります

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c psi4 psi4 python=3.10
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

--2023-07-05 06:15:42--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2023-07-05 06:15:42--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73134376 (70M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  69.75M   187MB/s    in 0.4s    

2023-07-05 06:15:43 (187 MB/s) - ‘Miniconda3-latest-Linux

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import psi4

print(f'current time: {datetime.datetime.now()}')
print(f'python version:\n{sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'psi4 version: {psi4.__version__}')

current time: 2023-07-05 06:19:41.480948
python version:
3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
numpy version: 1.22.4
pandas version: 1.5.3
psi4 version: 1.7


#### 計算資源の設定

In [ ]:
# 計算資源の確認（CPU, RAM）
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.206
cache size	: 56320 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4400.41
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
!cat /proc/meminfo

MemTotal:       53474568 kB
MemFree:        40907528 kB
MemAvailable:   51926512 kB
Buffers:          209344 kB
Cached:         11009344 kB
SwapCached:            0 kB
Active:          1195516 kB
Inactive:       10586308 kB
Active(anon):       1036 kB
Inactive(anon):   563448 kB
Active(file):    1194480 kB
Inactive(file): 10022860 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               840 kB
Writeback:             0 kB
AnonPages:        558580 kB
Mapped:           450440 kB
Shmem:              1348 kB
KReclaimable:     430404 kB
Slab:             510868 kB
SReclaimable:     430404 kB
SUnreclaim:        80464 kB
KernelStack:        6192 kB
PageTables:        11644 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    26737284 kB
Committed_AS:    2975204 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       10160 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
n_cpu = os.cpu_count()

In [ ]:
ram = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES') / (1024 ** 3)

In [ ]:
# 環境に応じて計算資源を設定
psi4.set_num_threads(n_cpu)
psi4.set_memory(f'{ram * 0.9: .0f}GB')

46000000000

### 3.3 水分子のエネルギー計算をしてみよう

#### 3.3.1 Psithon形式で計算を実行してみよう

In [ ]:
!which psi4

/usr/local/bin/psi4


In [ ]:
# h2o_psithon.inファイルの作成
!cat > h2o_psithon.in

molecule h2o {
0 1
O       -0.1176269719      0.7387773605      0.0000000000
H        0.8523730281      0.7387773605      0.0000000000
H       -0.4409567836      1.4770262439     -0.5397651517
}

energy('hf/sto-3g')
^C


In [ ]:
!ls -l

total 71448
-rw-r--r-- 1 root root      216 Jul  5 06:33 h2o_psithon.in
-rw-r--r-- 1 root root        0 Jul  5 06:31 h2o_psithon.log
-rw-r--r-- 1 root root     4228 Jul  5 06:31 h2o_psithon.out
-rwxr-xr-x 1 root root 73134376 Apr 24 14:26 Miniconda3-latest-Linux-x86_64.sh
drwxr-xr-x 1 root root     4096 Jun 29 13:30 sample_data
-rw-r--r-- 1 root root      922 Jul  5 06:31 timer.dat


In [ ]:
!cat h2o_psithon.in

molecule h2o {
0 1
O       -0.1176269719      0.7387773605      0.0000000000
H        0.8523730281      0.7387773605      0.0000000000
H       -0.4409567836      1.4770262439     -0.5397651517
}

energy('hf/sto-3g')


In [ ]:
# 計算の実行
!psi4 -i h2o_psithon.in -o h2o_psithon.out

In [ ]:
!grep 'Total Energy' h2o_psithon.out

                           Total Energy        Delta E     RMS |[F,P]|
    Total Energy =                        -74.9617284312669199


#### 3.3.2　PsiAPI形式での計算を実行してみよう

In [ ]:
# 計算ログファイルの設定
psi4.set_output_file('h2o_psiapi.log')

PosixPath('h2o_psiapi.log')

In [ ]:
# 水分子の構造定義
h2o = psi4.geometry('''
0 1
O       -0.1176269719      0.7387773605      0.0000000000
H        0.8523730281      0.7387773605      0.0000000000
H       -0.4409567836      1.4770262439     -0.5397651517
''')

In [ ]:
# エネルギー計算の実行
psi4.energy('hf/sto-3g', molecule=h2o)

INFO:psi4.driver.task_planner:PLANNING Atomic:  keywords={'function_kwargs': {}}
INFO:psi4.driver.driver:Compute energy(): method=hf, basis=sto-3g, molecule=default, nre=9.062772525153836
INFO:psi4.driver.driver:Return energy(): -74.96172843126848


-74.96172843126848